In [174]:
import pandas as pd
import re
from collections import Counter
from math import log

> Дано 2 csv-файла: один с текстом — <span style="color:green">articles.csv</span>, второй со стоп-словами — <span style="color:green">stopwords.csv</span>.
> 
> Необходимо в Jupyter-ноутбуке выполнить следующие пункты, используя SQLite или Pandas совместно с Python (очистку данных можно проводить также либо с помощью Python, либо используя SQL или Pandas).

## Шаг 1

> Прочитать файлы и создать над ними таблицы, где структуры таблиц:
> 
> 1. articles в виде:
> 
> **id: integer (nullable = true)**<br/>
> **text: string (nullable = true)**
> 
> 2. stopwords в виде:
> 
> **word: string (nullable = true)**

In [175]:
articles_df = pd.read_csv('data/articles.csv', delimiter=';', names=['id', 'text'], header=None)
stopwords_df = pd.read_csv('data/stopwords.csv', delimiter=';', names=['word'], header=None)

In [176]:
articles_df.head()

,id,text
0,1,Bradley Charles Cooper born January 5 1975 is ...
1,2,Cooper enrolled in the MFA program at the Acto...
2,3,Cooper found greater success with the romantic...
3,4,Labeled a sex symbol by the media Cooper was n...
4,5,Cooper was born on January 5 1975 in Abingto...


In [177]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 360 entries, 0 to 359
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      360 non-null    int64 
 1   text    360 non-null    object
dtypes: int64(1), object(1)
memory usage: 5.8+ KB


In [178]:
articles_df.describe()

,id
count,360.000000
mean,180.500000
std,104.067286
min,1.000000
25%,90.750000
50%,180.500000
75%,270.250000
max,360.000000


In [179]:
stopwords_df.head()

,word
0,x
1,y
2,your
3,yours
4,yourself


In [180]:
stopwords_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 733 entries, 0 to 732
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   word    733 non-null    object
dtypes: object(1)
memory usage: 5.9+ KB


In [181]:
stopwords_df.describe()

,word
count,733
unique,733
top,x
freq,1


> До выполнения задачи изначально обработать данные:
> 
>  - при парсинге отбросить все символы, которые не являются латинскими буквами;
>  - привести все слова к нижнему регистру;
>  - удалить все стоп-слова из articles с помощью таблицы stopwords.

In [182]:
stopwords = set(stopwords_df['word'])

In [183]:
def parse(str):
    str = re.sub('[^a-zA-Z ]', '', str)
    str = str.lower()
    return " ".join([x for x in str.split() if x not in stopwords])

In [184]:
articles_df['text'] = articles_df['text'].apply(lambda x: parse(x))

In [185]:
articles_df.head()

,id,text
0,1,bradley charles cooper born january american a...
1,2,cooper enrolled mfa program actors studio begi...
2,3,cooper found greater success romantic comedy s...
3,4,labeled sex symbol media cooper named people m...
4,5,cooper born january abington township near phi...


In [186]:
stopwords_df.head()

,word
0,x
1,y
2,your
3,yours
4,yourself


## Шаг 2

> Извлечь коллокации в тексте articles.csv. Это комбинации слов, которые часто встречаются вместе. Например, «smart boss» или «linings playbook». Чтобы найти совпадения, нужно использовать метрику NPMI (нормализованная точечная взаимная информация).

In [187]:
def npmi(a, b, ab):
    pmi = log(ab / (a * b))
    return -pmi / log(ab)

In [188]:
words_counter = Counter()
pairs_counter = Counter()
words_total = 0
pairs_total = 0

In [189]:
for row in articles_df['text']:
    words = row.split()
    words_total += len(words)
    pairs_total += len(words) - 1
    words_counter.update(words)
    pairs_counter.update(zip(words[:-1], words[1:]))

In [190]:
word_probs = {word: count / words_total for word, count in words_counter.items()}
pair_probs = {pair: count / pairs_total for pair, count in pairs_counter.items()}

> Таким образом, для каждой комбинации слов посчитать NPMI и вывести на экран TOP-50 самых популярных коллокаций, отсортированных по убыванию значения NPMI.
> 
> Комбинацию слов ab объединить пробелом.

In [191]:
npmi_vals = {pair: npmi(word_probs[pair[0]], word_probs[pair[1]], pair_prob) for pair, pair_prob in pair_probs.items()}
npmi_df = pd.DataFrame([(pair[0] + ' ' + pair[1], npmi) for pair, npmi in npmi_vals.items()], columns=['pairs', 'npmi'])

In [192]:
npmi_df.sort_values('npmi', ascending=False, inplace=True, ignore_index=True)
npmi_df.head(50)

,pairs,npmi
0,fish fry,1.005224
1,nightmare alley,1.005107
2,linings playbook,1.005107
3,los angeles,1.005107
4,guardians galaxy,1.004971
5,willy wanker,1.004805
6,iberian peninsula,1.004805
7,licorice pizza,1.004805
8,barack obama,1.004805
9,ella fitzgerald,1.004590
